In [1]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

# input에 한국어로 검색하면 이것을 컴퓨터 언어로 변환해줌
plusurl = urllib.parse.quote_plus(input("검색어를 입력하세요 : "))
 
# 페이지를 넘겨가면서 카운트 하기 위해 최초값을 1로 설정함.
page_num = 1
count = 1

i = input("몇 페이지까지를 크롤링 할까요? : ")

# 크롤링 범위를 제한하기 위해 lastpage를 정해줌.
lastpage = int(i) * 10 - 9

while page_num < lastpage + 1:
    url = f"https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query=%{plusurl}&sm=tab_pge&srchby=all&st=sim&where=post&start={page_num}"

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup)
    
    # find, select_one은 고른 태그의 최초값을 가져옴.
    # find_all, select는 고른 태그의 모든값을 리스트로 가져옴. 단 한개여도 리스트로.
    # find, find_all은 하위 태그를 지정할 때 ﻿find('div').find('a').find('img') 이렇게 해야하고
    # select_one("div a img")를 하면 됨. 다만 태그값에 " "가 있을 때 select는 하위값으로 인식하기 때문에 사용 못함.
    title = soup.find_all(class_='sh_blog_title')
    #print(title)


    print(f"{count}페이지 결과입니다.")
    # attrs는 태그의 title, id 속성값을 찾아줌. 속성값을 가져오는 게 아니라 출력함.
    for i in title:
        print(i.attrs['title'])
        print(i.attrs['href'])
    print()

    page_num += 10
    count += 1

검색어를 입력하세요 : 비건
몇 페이지까지를 크롤링 할까요? : 3
1페이지 결과입니다.

2페이지 결과입니다.

3페이지 결과입니다.



In [2]:
!pip install selenium

You should consider upgrading via the 'c:\users\samsung\appdata\local\programs\python\python36-32\python.exe -m pip install --upgrade pip' command.


In [5]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연
from tqdm.notebook import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

# 크롬 웹브라우저 실행
path = "./chromedriver.exe"
# .. 이전폴더
# . 현재폴더

driver = webdriver.Chrome(path)
driver.get('http://www.naver.com')

time.sleep(2) # 2초간 정지
# 정지코드를 넣는 이유 : 브라우저의 실행 속도보다 코드 진행이 빨라 오류가 날 수 있기 때문

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )

# 검색옵션 확인
item_li = driver.find_elements_by_css_selector('.option .txt')

for i in range(0, len(item_li)):
    print(item_li[i].text)

    # 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 5
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1


  # 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # 중간에 띄어쓰기는 .으로 대체
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
    print(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)
    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

df.to_excel("blog_url.xlsx")

1.크롤링할 키워드는 무엇입니까?: 비건
전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기
https://blog.naver.com/jihye7822?Redirect=Log&logNo=222459996618
https://blog.naver.com/eruriee?Redirect=Log&logNo=222455490120
https://blog.naver.com/thdud0212?Redirect=Log&logNo=222461854867
https://blog.naver.com/bushcherry1?Redirect=Log&logNo=222358070582
https://blog.naver.com/minju1004b?Redirect=Log&logNo=222436971165
https://blog.naver.com/eunbyeol11?Redirect=Log&logNo=222461144576
https://blog.naver.com/nahe0904?Redirect=Log&logNo=222456106495
https://blog.naver.com/mingming_93?Redirect=Log&logNo=222454781593
https://blog.naver.com/minuit_j?Redirect=Log&logNo=222461701037
https://blog.naver.com/bambi_showpu?Redirect=Log&logNo=222451228655
https://blog.naver.com/sunkjydbgn?Redirect=Log&logNo=222460458621
https://blog.naver.com/opcso806?Redirect=Log&logNo=222459970617
https://blog.naver.com/pavanel?Redirect=Log&logNo=222441023487
https://blog.naver.com/eruriee?Redirect=Log&logNo=222440620955
https://b

https://blog.naver.com/jjubu0691?Redirect=Log&logNo=222441791634
https://blog.naver.com/wldbs80899?Redirect=Log&logNo=222442050794
https://blog.naver.com/iamchocolat?Redirect=Log&logNo=222410872260
https://blog.naver.com/masi3333?Redirect=Log&logNo=222400007122
https://blog.naver.com/phj3000kr?Redirect=Log&logNo=222414533976
https://blog.naver.com/jyhwang317?Redirect=Log&logNo=222461616645
https://blog.naver.com/kan12?Redirect=Log&logNo=222454562359
https://blog.naver.com/nobokim?Redirect=Log&logNo=222452066838
https://blog.naver.com/kiddy28?Redirect=Log&logNo=222428711727
https://blog.naver.com/hyundorina?Redirect=Log&logNo=222436063636
https://blog.naver.com/morgunlee?Redirect=Log&logNo=222415151231
https://blog.naver.com/1ovest?Redirect=Log&logNo=222447620319
https://blog.naver.com/sielle83?Redirect=Log&logNo=222388225440
https://blog.naver.com/blingnas?Redirect=Log&logNo=222360297756
https://blog.naver.com/blueseashore?Redirect=Log&logNo=222458482230
https://blog.naver.com/dugubi2?

NameError: name 'head' is not defined

In [7]:
df.head(10)

,url,title
0,https://blog.naver.com/jihye7822?Redirect=Log&...,마스크트러블 관리 마녀공장 어성초 세럼 비건화장품으로
1,https://blog.naver.com/eruriee?Redirect=Log&lo...,"저알러지사료, 헤일로 가든오브비건 밥풀이랑 잘 맞아요!"
2,https://blog.naver.com/thdud0212?Redirect=Log&...,비건 천연 샴푸바 고체샴푸 로고나
3,https://blog.naver.com/bushcherry1?Redirect=Lo...,토니모리 비건아이라이너 롱래스팅 되네용!
4,https://blog.naver.com/minju1004b?Redirect=Log...,러블리 비건 딸기우유 핑크가방 JW PEI Maze 숄더백
5,https://blog.naver.com/eunbyeol11?Redirect=Log...,마스크피부트러블 비건화장품으로 관리해봄!
6,https://blog.naver.com/nahe0904?Redirect=Log&l...,비건 가방 JW PEI 신상 maze백 후기+할인코드
7,https://blog.naver.com/mingming_93?Redirect=Lo...,비건 베이킹클래스 내 손으로 만드는 비건빵 꼼므크리제
8,https://blog.naver.com/minuit_j?Redirect=Log&l...,마스크여드름 마녀공장 아워 비건 어성초 98 시카 세럼으로...
9,https://blog.naver.com/bambi_showpu?Redirect=L...,마녀공장 아워 비건 어성초 토너 & 세럼! 마스크 트러블...


In [10]:
df.to_excel("blog_url.xlsx")

In [9]:
!pip install openpyxl

You should consider upgrading via the 'c:\users\samsung\appdata\local\programs\python\python36-32\python.exe -m pip install --upgrade pip' command.


In [16]:
import numpy as np
import pandas as pd

url_load=pd.read_excel("blog_url.xlsx", engine='openpyxl')

#url_load = pd.read_excel("blog_url.xlsx")

num_list = len(url_load)

print(num_list)
url_load

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 30  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("./chromedriver.exe")  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')

180


  0%|          | 0/30 [00:00<?, ?it/s]

0 마스크트러블 관리 마녀공장 어성초 세럼 비건화장품으로
1 저알러지사료, 헤일로 가든오브비건 밥풀이랑 잘 맞아요!
2 비건 천연 샴푸바 고체샴푸 로고나 
3 토니모리 비건아이라이너 롱래스팅 되네용!
4 러블리 비건 딸기우유 핑크가방 JW PEI Maze 숄더백
5 마스크피부트러블 비건화장품으로 관리해봄!
6 비건 가방 JW PEI 신상 maze백 후기+할인코드
7 비건 베이킹클래스 내 손으로 만드는 비건빵 꼼므크리제
9 마녀공장 아워 비건 어성초 토너 & 세럼! 마스크 트러블 잠재우는 여름 피부관리
10 마스크 트러블 마녀공장 비건 어성초세럼으로 진정시켜요
11 좁쌀여드름 비건화장품, 마녀공장 어성초 98 시카세럼
12 올리브영 비건 에센스 데일리로!
13 비건에센스, 올리브영에서도 만날 수 있는 티엘스 콤부차
14 생봄(ST.BAUME)의 기초라인 베스트 세 가지를 소개드립니다, 진정한 비건 화장품입니다!!
15 이즈그린 LHA 비건 세트 민감성피부화장품
16 비건화장품 파이콜로지 n병째쓰는중
17 비건샴푸, 헬로가닉샴푸로 머릿결 관리하기 
18 비건화장품 토니모리 더 쇼킹 비건라이너 솔직후기!
19 대구 비건빵 침산동 빵집 <더순수브레드> 맛집수준
20 100% 비건 레시피 마녀공장 아워 비건 세럼 & 토너로 여름 민감피부 진정시키기!
22 슬로우 비건화장품 에필로우 아티초크 진정 앰플
23 토니모리 더 쇼킹 비건라이너, 더 쇼킹카라랑 같이써봤어
25 로고나 비건 천연 샴푸바 고체샴푸
26 티엘스 콤부차 비건에센스 순하네
27 아워 비건 어성초 98 시카 세럼 & 어성초 시카 토너,100% 비건 레시피 민감 피부 진정템 추천
28 비건패션 가방 JW PEI 가비백 후기 & 해외직구 할인코드 공유
29 마스크피부트러블 진정에는 비건화장품 어성초 세럼 추천
수집한 글 갯수:  27
{0: {'title': '마스크트러블 관리 마녀공장 어성초 세럼 비건화장품으로', 'nickname': '밤곰이\n(jihye7822)', 'datetime': '2021. 8.

In [12]:
!pip install xlrd

You should consider upgrading via the 'c:\users\samsung\appdata\local\programs\python\python36-32\python.exe -m pip install --upgrade pip' command.
